# Data Exploration

**Purpose**: Explore ingested data from ClickHouse before training

This notebook helps you understand:
- Data volume and distribution
- Time series patterns
- Class balance
- Missing values
- Feature correlations

## Setup

In [ ]:
import sys
sys.path.insert(0, '../')

from packages.training import FeatureExtractor, FeatureBuilder, ModelTrainer
from packages.storage import ClientFactory, get_connection_params
from notebook_utils import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger
from datetime import datetime, timedelta

setup_plotting()

## Configuration

In [ ]:
NETWORK = 'ethereum'
START_DATE = '2024-01-01'
END_DATE = '2024-03-31'
WINDOW_DAYS = 7

print(f"Network: {NETWORK}")
print(f"Date Range: {START_DATE} to {END_DATE}")
print(f"Window: {WINDOW_DAYS} days")

## Connect to ClickHouse

In [ ]:
connection_params = get_connection_params(NETWORK)
client_factory = ClientFactory(connection_params)

print("Connected to ClickHouse")
print(f"Database: {connection_params['database']}")

## Query Raw Alerts

In [ ]:
with client_factory.client_context() as client:
    query = f"""
    SELECT
        alert_id,
        created_at,
        severity,
        source,
        alert_hash,
        addresses
    FROM raw_alerts
    WHERE network = '{NETWORK}'
        AND created_at BETWEEN '{START_DATE}' AND '{END_DATE}'
    ORDER BY created_at
    """
    
    alerts_df = client.query_df(query)

print(f"Total alerts: {len(alerts_df):,}")
alerts_df.head()

## Alert Statistics

In [ ]:
print("Alert Statistics:")
print(f"Total Alerts: {len(alerts_df):,}")
print(f"Date Range: {alerts_df['created_at'].min()} to {alerts_df['created_at'].max()}")
print(f"Unique Sources: {alerts_df['source'].nunique()}")
print(f"\nSeverity Distribution:")
print(alerts_df['severity'].value_counts())

## Time Series Analysis

In [ ]:
alerts_df['date'] = pd.to_datetime(alerts_df['created_at']).dt.date
daily_counts = alerts_df.groupby('date').size()

plt.figure(figsize=(14, 6))
daily_counts.plot(kind='line', marker='o')
plt.title('Alert Volume Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Alerts')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Severity Distribution

In [ ]:
severity_counts = alerts_df['severity'].value_counts()

plt.figure(figsize=(10, 6))
severity_counts.plot(kind='bar')
plt.title('Alert Severity Distribution')
plt.xlabel('Severity')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nSeverity Percentages:")
print((severity_counts / len(alerts_df) * 100).round(2))

## Query Raw Features

In [ ]:
with client_factory.client_context() as client:
    query = f"""
    SELECT *
    FROM raw_features
    WHERE network = '{NETWORK}'
        AND timestamp BETWEEN '{START_DATE}' AND '{END_DATE}'
    LIMIT 10000
    """
    
    features_df = client.query_df(query)

print(f"Total features sampled: {len(features_df):,}")
features_df.head()

## Feature Data Quality

In [ ]:
print("Missing Values:")
missing = features_df.isnull().sum()
missing_pct = (missing / len(features_df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

print("\nData Types:")
print(features_df.dtypes)

## Numeric Feature Distributions

In [ ]:
numeric_cols = features_df.select_dtypes(include=[np.number]).columns.tolist()
if 'address' in numeric_cols:
    numeric_cols.remove('address')

print(f"Numeric features: {len(numeric_cols)}")
print(numeric_cols[:10])

if len(numeric_cols) > 0:
    features_df[numeric_cols[:6]].describe()

In [ ]:
if len(numeric_cols) >= 6:
    plot_feature_distributions(features_df, numeric_cols[:6])
    plt.show()

## Query Raw Clusters

In [ ]:
with client_factory.client_context() as client:
    query = f"""
    SELECT
        cluster_id,
        created_at,
        alert_count,
        address_count,
        severity_distribution
    FROM raw_clusters
    WHERE network = '{NETWORK}'
        AND created_at BETWEEN '{START_DATE}' AND '{END_DATE}'
    LIMIT 5000
    """
    
    clusters_df = client.query_df(query)

print(f"Total clusters sampled: {len(clusters_df):,}")
clusters_df.head()

## Cluster Analysis

In [ ]:
print("Cluster Statistics:")
print(f"Total Clusters: {len(clusters_df):,}")
print(f"\nAlert Count Distribution:")
print(clusters_df['alert_count'].describe())
print(f"\nAddress Count Distribution:")
print(clusters_df['address_count'].describe())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(clusters_df['alert_count'], bins=50, edgecolor='black')
axes[0].set_title('Cluster Alert Count Distribution')
axes[0].set_xlabel('Alert Count')
axes[0].set_ylabel('Frequency')
axes[0].set_yscale('log')

axes[1].hist(clusters_df['address_count'], bins=50, edgecolor='black')
axes[1].set_title('Cluster Address Count Distribution')
axes[1].set_xlabel('Address Count')
axes[1].set_ylabel('Frequency')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

## Correlation Analysis

In [ ]:
if len(numeric_cols) >= 5:
    sample_features = features_df[numeric_cols[:10]].copy()
    sample_features = sample_features.fillna(0)
    
    plot_correlation_matrix(sample_features, figsize=(10, 8))
    plt.show()

## Conclusions

**Key Findings**:

1. **Data Volume**: Review total alerts, features, and clusters
2. **Time Patterns**: Check for seasonality or trends
3. **Severity Distribution**: Understand class balance
4. **Data Quality**: Identify missing values and outliers
5. **Correlations**: Detect highly correlated features

**Next Steps**:
- Proceed to Feature Analysis notebook
- Address data quality issues if found
- Consider class balancing strategies if needed